In [3]:
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go

import tensorflow.compat.v1 as tf
from tensorflow.contrib import cluster_resolver as contrib_cluster_resolver
from tensorflow.contrib import tpu as contrib_tpu
import sys
sys.path.append("./ALBERT/")
from ALBERT import *
import tensorflow_hub as hub
from classifier_utils import DataProcessor, InputExample,file_based_convert_examples_to_features,file_based_input_fn_builder,model_fn_builder

In [2]:
TRAIN_FILE = Path("./Data/SemEval2012_train.csv")
TEST_FILE = Path("./Data/SemEval2012_test.csv")

In [3]:
train_df = pd.read_csv(TRAIN_FILE)
test_df = pd.read_csv(TEST_FILE)

In [4]:
train_df = train_df.reset_index(drop = True)
train_df.columns.to_list()

['sent1', 'sent2', 'score']

In [5]:
for i,j in zip(train_df.index, train_df['sent1'].to_list()):
    print(i,j)
    break

0 But other sources close to the sale said Vivendi was keeping the door open to further bids and hoped to see bidders interested in individual assets team up.


In [6]:
class SemEval(DataProcessor):
    def get_train_examples(self, df):
        return self._create_examples(df)
    def get_labels(self):
        return ["0","1"]
    def _create_examples(self, semeval_df):
        examples = []
        guids = train_df.index
        semeval_df['score'] = semeval_df['score'].astype(float)
        sent1 = train_df['sent1'].to_list()
        sent2 = train_df['sent2'].to_list()
        score = train_df['score'].to_list()
        for guid, text_a, text_b, label in zip(guids, sent1, sent2, score):
            examples.append(InputExample(guid=guid, text_a = text_a, text_b=text_b, label=str(int(label))))
        return examples

In [7]:
processor = SemEval(use_spm = True, do_lower_case = True)

In [8]:
classifier_utilslabel_list = processor.get_labels()

In [9]:
albert_hub = "https://tfhub.dev/google/albert_base/3"

In [10]:
from fine_tuning_utils import create_vocab
tokenizer = create_vocab(vocab_file=None, do_lower_case=False, spm_model_file=True, hub_module="https://tfhub.dev/google/albert_base/3")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:loading sentence piece model


INFO:tensorflow:loading sentence piece model


In [11]:
train_examples = processor.get_train_examples(train_df)

In [12]:
train_file = "./Data/train.tf_record"

In [13]:
label_list = ['0','1']

In [14]:
file_based_convert_examples_to_features(train_examples, label_list, 500, tokenizer, train_file, "SemEval")

INFO:tensorflow:Writing example 0 of 2234


INFO:tensorflow:Writing example 0 of 2234


INFO:tensorflow:using sentence piece tokenzier.


INFO:tensorflow:using sentence piece tokenzier.


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:using sentence piece tokenzier.


INFO:tensorflow:using sentence piece tokenzier.


INFO:tensorflow:using sentence piece tokenzier.


INFO:tensorflow:using sentence piece tokenzier.


INFO:tensorflow:using sentence piece tokenzier.


In [15]:
train_input_fn = file_based_input_fn_builder(input_file = train_file, seq_length=500, is_training=True, drop_remainder=True, task_name="SemEval", use_tpu = False, bsz=32)

In [ ]:
is_per_host = contrib_tpu.InputPipelineConfig.PER_HOST_V2
iterations_per_loop = 1000

run_config = contrib_tpu.RunConfig(
      cluster=None,
      master=None,
      model_dir="./Data/checkpoints/",
      save_checkpoints_steps=1000,
      keep_checkpoint_max=0,
      tpu_config=contrib_tpu.TPUConfig(
          iterations_per_loop=iterations_per_loop,
          num_shards=8,
          per_host_input_for_training=is_per_host))

model_fn = model_fn_builder(
      albert_config=None,
      num_labels=len(label_list),
      init_checkpoint=None,
      learning_rate=5e-5,
      num_train_steps=1,
      num_warmup_steps=0,
      use_tpu=False,
      use_one_hot_embeddings=False,
      task_name='SemEval',
      hub_module=albert_hub,
      optimizer='adamw')

estimator = contrib_tpu.TPUEstimator(
      use_tpu=False,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=2,
      eval_batch_size=8,
      predict_batch_size=32)

cached_dir = "./Data/cache"

tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", 2)
tf.logging.info("  Num steps = %d", 1)

estimator.train(input_fn=train_input_fn, max_steps=1000)



INFO:tensorflow:Using config: {'_model_dir': './Data/checkpoints/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 0, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa5104075f8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=No

INFO:tensorflow:Using config: {'_model_dir': './Data/checkpoints/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 0, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa5104075f8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=No

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:***** Running training *****


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 0 into ./Data/checkpoints/model.ckpt.


INFO:tensorflow:Saving checkpoints for 0 into ./Data/checkpoints/model.ckpt.


In [4]:
tf.__version__

'1.15.2'